# 19BCE123: JASH MAVANI

## Practical 1

Kaggle: Titanic – Machine Learning from Disaster
(https://www.kaggle.com/c/titanic)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import

In [2]:
import numpy as np
import pandas as pd 


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
ans = pd.read_csv("../input/titanic/gender_submission.csv")

In [5]:
train

In [6]:
train.isnull().sum()

**Handling Null values**

In [7]:
print(train['Embarked'].value_counts())
print(round(train['Age'].mean(),2))

In [8]:
train['Age'] = train['Age'].fillna(round(train['Age'].mean(),2))
train['Embarked'] = train['Embarked'].fillna('S')
train = train.drop('Cabin', 1)
train = train.drop('Name',1)

In [9]:
train.shape

In [10]:
test

In [11]:
test.isnull().sum()

In [12]:
print(round(test['Age'].mean(),2))
print(round(test['Fare'].mean(),3))

In [13]:
test['Age'] = test['Age'].fillna(round(test['Age'].mean(),2))
test['Fare'] = test['Fare'].fillna(round(test['Fare'].mean(),3))
test = test.drop('Cabin', 1)
test = test.drop('Name',1)

In [14]:
test.shape

In [15]:
train['Who'] = train['Sex'].apply(lambda x :'male' if x=='male' else 'female' if x=='female' else ' ' )

train

In [16]:
for i in range(0,len(train)) :
    if train['Age'][i]<=15 :
        train['Who'][i] = 'child'
        
train

In [17]:
test['Who'] = test['Sex'].apply(lambda x :'male' if x=='male' else 'female' if x=='female' else ' ' )

for i in range(0,len(test)) :
    if test['Age'][i]<=15 :
        test['Who'][i] = 'child'

test

In [18]:
sum = 0
for i in range(len(train)) :
    if (train['Age'][i]>=2 and train['Age'][i]<=30 and train['Sex'][i]=='female' and train['Survived'][i]==1) :
        sum = sum + 1
        
print(sum)

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
 
sns.countplot(x =train['Sex'],hue = train['Survived'], data = train)
 
plt.show()

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
 
sns.countplot(x =train['Who'],hue = train['Survived'], data = train)
 
plt.show()

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
 
sns.countplot(x =train['Pclass'],hue = train['Survived'], data = train)
 
plt.show()

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
 
sns.countplot(x =train['Parch'],hue = train['Survived'], data = train)
 
plt.show()

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt
 
sns.countplot(x =train['Embarked'],hue = train['Survived'], data = train)
 
plt.show()

In [24]:
Y_train = train['Survived']
X_train = train.drop('Survived', 1)

print(X_train.shape)
print(Y_train)

In [25]:
X_test = test
test.shape

In [26]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

le = LabelEncoder()
X_train = X_train.apply(le.fit_transform)
print(X_train)

X_test = X_test.apply(le.fit_transform)
print(X_test)

In [27]:
selected_columns1 = X_train[["Pclass", "Sex","Embarked", "SibSp", "Parch", "Who"]]
selected_columns2 = X_test[["Pclass", "Sex", "Embarked", "SibSp", "Parch", "Who"]]

X_train = selected_columns1.copy()
X_test = selected_columns2.copy()

print(Y_train)

print(X_test)

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)
scaler.transform(X_train)

scaler.fit(X_test)
scaler.transform(X_test)

In [29]:
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(n_neighbors=5)

# knn.fit(X_train, Y_train)
# Y_pred = knn.predict(X_test)

In [30]:
X_test

In [31]:
# from sklearn.metrics import accuracy_score

# accuracy_score(y_true, y_pred)

# print(len(ans['PassengerId']))
# print(len(Y_pred))

In [32]:
from sklearn.tree import DecisionTreeClassifier

clf_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 125,max_depth=10, min_samples_leaf=5)

clf_gini.fit(X_train, Y_train)

Y_pred = clf_gini.predict(X_test)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': max_depth,
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

print(random_grid)

In [34]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train, Y_train)
Y_pred = rf_random.predict(X_test)

In [35]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import RandomizedSearchCV

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=2)

# model.fit(X_train, Y_train)
# Y_pred = model.predict(X_test)

In [36]:
data = []
for i in range(0,418) :
    one = []
    one.append(i+892)
    one.append(Y_pred[i])
    data.append(one)
    
df = pd.DataFrame(data, columns = ['PassengerId', 'Survived'])
df

In [ ]:
df.to_csv('/kaggle/working/ans.csv',index=False)